## Pressure velocity at 500 hpa

$\omega_{500}$ - used to calculate ascent area and descent area

ascent area - mean of the fraction of gridboxes with $\omega_{500} < 0$ over a given month  
descent area - mean of the fraction of gridboxes with $\omega_{500} > 0$, over a given month

In [1]:
# import intake

In [2]:
# experiment = 'historical'
# period=slice('1970-01','1999-12')
# ensemble = 'r1i1p1'
# model = 'GFDL-CM3'

# ds_dict = intake.cat.nci['esgf'].cmip5.search(
#                                 model_id = model, 
#                                 experiment = experiment,
#                                 time_frequency = 'day', 
#                                 realm = 'atmos', 
#                                 ensemble = ensemble, 
#                                 variable= 'pr').to_dataset_dict()

In [3]:
# list(intake.cat.nci['esgf'])

In [4]:
# experiment = 'historical'
# period=slice('1970-01','1999-12')
# ensemble = 'r1i1p1'
# model = 'GFDL-CM3'

# a = intake.cat.nci['esgf'].cmip5

In [5]:
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt

In [27]:
# folder = '/g/data/al33/replicas/CMIP5/combined/NOAA-GFDL/GFDL-CM3/historical/day/atmos/day/r1i1p1/v20120227/wap'
# fileName = 'wap_day_GFDL-CM3_historical_r1i1p1_19350101-19391231.nc'

folder = '/g/data/al33/replicas/CMIP5/combined/NOAA-GFDL/GFDL-CM3/rcp85/day/atmos/day/r1i1p1/v20120227/wap'
fileName = 'wap_day_GFDL-CM3_rcp85_r1i1p1_20060101-20101231.nc'
path = folder + '/' + fileName

ds = xr.open_dataset(path)
ds

<xarray.Dataset>
Dimensions:     (plev: 8, lat: 90, bnds: 2, lon: 144, time: 1825)
Coordinates:
  * plev        (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * lat         (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
  * lon         (lon) float64 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * time        (time) object 2006-01-01 12:00:00 ... 2010-12-31 12:00:00
Dimensions without coordinates: bnds
Data variables:
    lat_bnds    (lat, bnds) float64 ...
    lon_bnds    (lon, bnds) float64 ...
    time_bnds   (time, bnds) object ...
    average_T1  (time) datetime64[ns] ...
    average_T2  (time) datetime64[ns] ...
    average_DT  (time) timedelta64[ns] ...
    wap         (time, plev, lat, lon) float32 ...
Attributes: (12/27)
    title:                  NOAA GFDL GFDL-CM3, RCP8.5 (run 1) experiment out...
    institute_id:           NOAA GFDL
    source:                 GFDL-CM3 2010 atmosphere: AM3 (AM3p9,C48L48); sea...
    contact:                gfdl.climate.model.info@noaa.gov
    project_id:             CMIP5
    table_id:               Table day (31 Jan 2011)
    ...                     ...
    parent_experiment_id:   historical
    physics_version:        1
    product:                output1
    institution:            NOAA GFDL(201 Forrestal Rd, Princeton, NJ, 08540)
    history:                File was processed by fremetar (GFDL analog of CM...
    parent_experiment_rip:  r1i1p1

In [9]:
import intake
import scipy
import xarray as xr
import xesmf as xe



def regrid_conserv(ds_in, haveDsOut, path='/g/data/k10/cb4968/data/cmip5/FGOALS-g2/FGOALS-g2_ds_regid_historical.nc', modelDsOut='FGOALS-g2'):

    if haveDsOut:
        ds_out = xr.open_dataset(path)
        regrid = xe.Regridder(ds_in, ds_out, 'conservative', periodic=True)
    
    else:
        ds_dict = intake.cat.nci['esgf'].cmip5.search(
                                        model_id = modelDsOut, 
                                        experiment = 'historical',
                                        time_frequency = 'day', 
                                        realm = 'atmos', 
                                        ensemble = 'r1i1p1', 
                                        variable= 'pr').to_dataset_dict()

        ds_regrid = ds_dict[list(ds_dict.keys())[-1]].sel(time='1970-01-01', lon=slice(0,360),lat=slice(-30,30))
        ds_regrid.to_netcdf(path)

        ds_out = xr.open_dataset(path)
        regrid = xe.Regridder(ds_in, ds_out, 'conservative', periodic=True)
        
    return regrid(ds_in)

In [10]:
ds_orig = ds
haveDsOut = True
ds_hus = regrid_conserv(ds_orig, haveDsOut) # path='', model ='')

In [28]:
# fileList = [
#     'wap_day_GFDL-CM3_historical_r1i1p1_19700101-19741231.nc',
#     'wap_day_GFDL-CM3_historical_r1i1p1_19750101-19791231.nc',
#     'wap_day_GFDL-CM3_historical_r1i1p1_19800101-19841231.nc',
#     'wap_day_GFDL-CM3_historical_r1i1p1_19850101-19891231.nc',
#     'wap_day_GFDL-CM3_historical_r1i1p1_19900101-19941231.nc',
#     'wap_day_GFDL-CM3_historical_r1i1p1_19950101-19991231.nc'
#     ]



fileList = [
    'wap_day_GFDL-CM3_rcp85_r1i1p1_20660101-20701231.nc',
    'wap_day_GFDL-CM3_rcp85_r1i1p1_20710101-20751231.nc',
    'wap_day_GFDL-CM3_rcp85_r1i1p1_20760101-20801231.nc',
    'wap_day_GFDL-CM3_rcp85_r1i1p1_20810101-20851231.nc',
    'wap_day_GFDL-CM3_rcp85_r1i1p1_20860101-20901231.nc',
    'wap_day_GFDL-CM3_rcp85_r1i1p1_20910101-20951231.nc',
    'wap_day_GFDL-CM3_rcp85_r1i1p1_20960101-21001231.nc'
]


In [29]:
path_full = []
for file in fileList:
    path_full = np.append(path_full, os.path.join(folder, file))

In [30]:
path_full

array(['/g/data/al33/replicas/CMIP5/combined/NOAA-GFDL/GFDL-CM3/rcp85/day/atmos/day/r1i1p1/v20120227/wap/wap_day_GFDL-CM3_rcp85_r1i1p1_20660101-20701231.nc',
       '/g/data/al33/replicas/CMIP5/combined/NOAA-GFDL/GFDL-CM3/rcp85/day/atmos/day/r1i1p1/v20120227/wap/wap_day_GFDL-CM3_rcp85_r1i1p1_20710101-20751231.nc',
       '/g/data/al33/replicas/CMIP5/combined/NOAA-GFDL/GFDL-CM3/rcp85/day/atmos/day/r1i1p1/v20120227/wap/wap_day_GFDL-CM3_rcp85_r1i1p1_20760101-20801231.nc',
       '/g/data/al33/replicas/CMIP5/combined/NOAA-GFDL/GFDL-CM3/rcp85/day/atmos/day/r1i1p1/v20120227/wap/wap_day_GFDL-CM3_rcp85_r1i1p1_20810101-20851231.nc',
       '/g/data/al33/replicas/CMIP5/combined/NOAA-GFDL/GFDL-CM3/rcp85/day/atmos/day/r1i1p1/v20120227/wap/wap_day_GFDL-CM3_rcp85_r1i1p1_20860101-20901231.nc',
       '/g/data/al33/replicas/CMIP5/combined/NOAA-GFDL/GFDL-CM3/rcp85/day/atmos/day/r1i1p1/v20120227/wap/wap_day_GFDL-CM3_rcp85_r1i1p1_20910101-20951231.nc',
       '/g/data/al33/replicas/CMIP5/combined/NOAA-GF

In [31]:
ds_all = xr.open_mfdataset(path_full, combine='by_coords')

In [32]:
ds_all

<xarray.Dataset>
Dimensions:     (plev: 8, lat: 90, time: 12775, bnds: 2, lon: 144)
Coordinates:
  * plev        (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * lat         (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
  * lon         (lon) float64 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * time        (time) object 2066-01-01 12:00:00 ... 2100-12-31 12:00:00
Dimensions without coordinates: bnds
Data variables:
    lat_bnds    (time, lat, bnds) float64 dask.array<chunksize=(1825, 90, 2), meta=np.ndarray>
    lon_bnds    (time, lon, bnds) float64 dask.array<chunksize=(1825, 144, 2), meta=np.ndarray>
    time_bnds   (time, bnds) object dask.array<chunksize=(1825, 2), meta=np.ndarray>
    average_T1  (time) datetime64[ns] dask.array<chunksize=(1825,), meta=np.ndarray>
    average_T2  (time) datetime64[ns] dask.array<chunksize=(1825,), meta=np.ndarray>
    average_DT  (time) timedelta64[ns] dask.array<chunksize=(1825,), meta=np.ndarray>
    wap         (time, plev, lat, lon) float32 dask.array<chunksize=(1825, 8, 90, 144), meta=np.ndarray>
Attributes: (12/27)
    title:                  NOAA GFDL GFDL-CM3, RCP8.5 (run 1) experiment out...
    institute_id:           NOAA GFDL
    source:                 GFDL-CM3 2010 atmosphere: AM3 (AM3p9,C48L48); sea...
    contact:                gfdl.climate.model.info@noaa.gov
    project_id:             CMIP5
    table_id:               Table day (31 Jan 2011)
    ...                     ...
    parent_experiment_id:   historical
    physics_version:        1
    product:                output1
    institution:            NOAA GFDL(201 Forrestal Rd, Princeton, NJ, 08540)
    history:                File was processed by fremetar (GFDL analog of CM...
    parent_experiment_rip:  r1i1p1

In [33]:
ds_orig = ds_all.sel(lat=slice(-35,35))
# haveDsOut = True
# ds_hus = regrid_conserv(ds_orig, haveDsOut) # path='', model ='')

In [25]:
def save_file(dataSet, folder, fileName):
    
    os.makedirs(folder, exist_ok=True)
    path = folder + '/' + fileName

    if os.path.exists(path):
        os.remove(path)    
    
    dataSet.to_netcdf(path)

In [26]:
folder = '/g/data/k10/cb4968/data/cmip5/ds'
fileName = 'GFDL-CM3_wap_rcp85.nc'
save_file(ds_orig, folder, fileName)